<center>
<img src="https://miro.medium.com/max/1400/1*XpCUS2g8yYsEejhH1vKnYg.png"/>

## <center> Специализация "Машинное обучение и анализ данных"
# <center> Итоговый проект:<br>Идентификация пользователей по посещенным веб-страницам
## <div style="text-align: right">Автор: Пётр Зюзиков</div>

### Исходные данные:
В нашем распоряжении есть база данных пользователей и их рабочих сессий в Интернете: последовательностей из нескольких веб-сайтов, посещенных подряд. Данные взяты из [соревнования "Catch Me If You Can"](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2).
### Задача:
Найти наиболее подходящий алгоритм и обучить его на этих данных, чтобы максимально успешно определять, принадлежит ли сессия определённому пользователю (бинарная классификация). Решаем для пользователя Элис (Alice).
**Метрика: ROC AUC**
### Определение успеха проекта:
1) Написать стабильно работающий код модели.<br>
2) Улучшить метрику в сравнении с эталонным решением.<br>
Личная цель: Войти в топ 5% лучших решений среди 4000 студентов за всё время или максимально подойти к этому рубежу.

# <center>Итоги
### ROC AUC
**Public: 0.95154, cross-validation: 0.00000. Эталон: 0.94958, 0.91623.**
### Топ 5% лидерборда (ссылка)
**Скриншот лидерборда**

### Краткое описание процесса решения:
Начну с вещи, которую я сознательно не стал делать: leaderboard probing. Из-за текущего формата соревнования доступен только публичный лидерборд и нет возможности узнать итоговые результаты на приватной части тестов. И это открывает широкие возможности для махинаций с файлом ответов. Вместо этого реализована кросс-валидация с правильной зависимостью от времени, и я ставлю её значение на один уровень важности с результатами публичного лидерборда.

При этом к исследованию Data Leaks у меня совсем другое отношение. **продолжение следует**

### <center>Шаблон решения
Этот исходный код — [от организаторов](https://www.kaggle.com/kashnitsky/alice-template-for-your-solution). Задания курса в течение предыдущих 6 недель включают в себя самостоятельную реализацию кода всех частей этого решения. Ноутбуки c моими решениями можно посмотреть в папке Preparation.

In [175]:
import os
import pickle
import numpy as np
import pandas as pd
import time
from contextlib import contextmanager
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression


PATH_TO_DATA = 'kaggle'
AUTHOR = 'EyeShield77'
SEED = 17
N_JOBS = 4
NUM_TIME_SPLITS = 10    # for time-based cross-validation
SITE_NGRAMS = (1, 5)    # site ngrams for "bag of sites"
MAX_FEATURES = 50000    # max features for "bag of sites"
BEST_LOGIT_C = 5.45559  # precomputed tuned C for logistic regression
 

# reporting running times
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')


def prepare_sparse_features(path_to_train, path_to_test, path_to_site_dict,
                           vectorizer_params):
    times = ['time%s' % i for i in range(1, 11)]
    train_df = pd.read_csv(path_to_train,
                       index_col='session_id', parse_dates=times)
    test_df = pd.read_csv(path_to_test,
                      index_col='session_id', parse_dates=times)

    # Sort the data by time
    train_df = train_df.sort_values(by='time1')
    
    # read site -> id mapping provided by competition organizers 
    with open(path_to_site_dict, 'rb') as f:
        site2id = pickle.load(f)
    # create an inverse id _> site mapping
    id2site = {v:k for (k, v) in site2id.items()}
    # we treat site with id 0 as "unknown"
    id2site[0] = 'unknown'
    
    # Transform data into format which can be fed into TfidfVectorizer
    # This time we prefer to represent sessions with site names, not site ids. 
    # It's less efficient but thus it'll be more convenient to interpret model weights.
    sites = ['site%s' % i for i in range(1, 11)]
    train_sessions = train_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    test_sessions = test_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    # we'll tell TfidfVectorizer that we'd like to split data by whitespaces only 
    # so that it doesn't split by dots (we wouldn't like to have 'mail.google.com' 
    # to be split into 'mail', 'google' and 'com')
    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_sessions)
    X_test = vectorizer.transform(test_sessions)
    y_train = train_df['target'].astype('int').values
    
    # we'll need site visit times for further feature engineering
    train_times, test_times = train_df[times], test_df[times]
    
    return X_train, X_test, y_train, vectorizer, train_times, test_times


def add_features(times, X_sparse):
    hour = times['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <= 6)).astype('int').values.reshape(-1, 1)
    sess_duration = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[s]')\
		   .astype('int').values.reshape(-1, 1)
    day_of_week = times['time1'].apply(lambda t: t.weekday()).values.reshape(-1, 1)
    month = times['time1'].apply(lambda t: t.month).values.reshape(-1, 1) 
    year_month = times['time1'].apply(lambda t: 100 * t.year + t.month).values.reshape(-1, 1) / 1e5

    X = hstack([X_sparse, morning, day, evening, night, sess_duration, day_of_week, month, year_month])
    return X


with timer('Building sparse site features'):
    X_train_sites, X_test_sites, y_train, vectorizer, train_times, test_times = \
        prepare_sparse_features(
            path_to_train=os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
            path_to_test=os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
            path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl'),
            vectorizer_params={'ngram_range': SITE_NGRAMS,
                               'max_features': MAX_FEATURES,
                               'tokenizer': lambda s: s.split()})


with timer('Building additional features'):
    X_train_final = add_features(train_times, X_train_sites)
    X_test_final = add_features(test_times, X_test_sites)


with timer('Cross-validation'):
    time_split = TimeSeriesSplit(n_splits=NUM_TIME_SPLITS)
    logit = LogisticRegression(random_state=SEED, solver='liblinear')

    c_values = [BEST_LOGIT_C]

    logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=N_JOBS, cv=time_split, verbose=1)
    logit_grid_searcher.fit(X_train_final, y_train)
    print('CV score', logit_grid_searcher.best_score_)


with timer('Test prediction and submission'):
    test_pred = logit_grid_searcher.predict_proba(X_test_final)[:, 1]
    pred_df = pd.DataFrame(test_pred, index=np.arange(1, test_pred.shape[0] + 1),
                       columns=['target'])
    pred_df.to_csv(f'submission_alice_{AUTHOR}.csv', index_label='session_id')

[Building sparse site features] done in 27 s
[Building additional features] done in 6 s
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   17.5s finished


CV score 0.9162289070077999
[Cross-validation] done in 26 s
[Test prediction and submission] done in 0 s


## Улучшаем решение

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки) и посмотрим на них.**

In [180]:
PATH_TO_DATA = 'kaggle/'
times = ['time%d' % i for i in range(1, 11)]
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id', parse_dates=times)
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id', parse_dates=times)
print('train_df.shape =', train_df.shape)
print('test_df.shape =', test_df.shape)
print('Разделение на классы в тренировочной выборке:')
print('Элис:', train_df['target'].value_counts()[1], ', остальные:', train_df['target'].value_counts()[0])

train_df.shape = (253561, 21)
test_df.shape = (82797, 20)
Разделение на классы в тренировочной выборке:
Элис: 2297 , остальные: 251264


In [5]:
train_df.head(3)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0


In [6]:
test_df.head(3)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36


В выборках мы видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут.

**Настоящие исходные данные.** Но это не все данные, которые нам доступны. Есть ещё архив train.zip с сессиями каждого пользователя в формате timestamp & site: время и посещённый сайт. И именно эти данные предобработали в тренировочную выборку, и по такому же алгоритму тестовые данные обработали в тестовую.
Сессии делили по полные по 10 сайтов до момента, пока не появлялся пропуск более получаса. На этом моменте записывали "короткую" сессию из 1-10 сайтов и NaN на остальных местах.

**Также давайте вспомним, как люди в 2013-2014 году выходят в Интернет.** Если кратко, то с компьютера ровно так же, как и сейчас: запускается браузер с окнами, которые остались с предыдущей сессии. В примере снизу на примере данных Элис мы видим, что за 5 секунд она открыла 9 сайтов. Подобные примеры мы увидим везде: и в тренировочных, и в тестовых данных.

In [190]:
alice_df = pd.read_csv('data/train/Alice_log.csv', parse_dates=['timestamp'])
print(f'Количество записей: {alice_df.shape[0]}')
alice_df[4:13]

Количество записей: 22769


,timestamp,site
4,2013-02-12 16:32:24,www.google.fr
5,2013-02-12 16:32:25,www.info-jeunes.net
6,2013-02-12 16:32:25,www.google.fr
7,2013-02-12 16:32:26,www.info-jeunes.net
8,2013-02-12 16:32:27,platform.twitter.com
9,2013-02-12 16:32:27,www.info-jeunes.net
10,2013-02-12 16:32:27,www.facebook.com
11,2013-02-12 16:32:28,www.info-jeunes.net
12,2013-02-12 16:32:29,twitter.com


Посмотрим, сколько на самом деле было получасовых промежутков между открытиями сайтов у Элис. А также 20, 15, 10 и 5-минутных.

In [191]:
alice_diff = (np.diff(alice_df['timestamp'])/np.timedelta64(1, 's'))
for i in [30, 20, 15, 10, 5]:
    print(f'{i:2.0f} min: {len(alice_diff[alice_diff > 60*i])}')

30 min: 34
20 min: 39
15 min: 48
10 min: 62
 5 min: 100


Если мы определим границу сессии в 5 минут, то ожидаем в среднем 22769/100 = 23 сайта за сессию.

Ещё посмотрим, насколько часто мы видим разницу не больше 1, 2, 3, 4 и 5 секунд между открытием сайтов.

In [10]:
for i in range(1, 6):
    print(f'{i} sec: {len(alice_diff[alice_diff <= i])}')

1 sec: 16841
2 sec: 18301
3 sec: 19009
4 sec: 19493
5 sec: 19877


Сколько раз минимум три сайта были открыты в течение 1й секунды?

In [232]:
TIME_DIFFERENCE = 2
NUMBER_OF_CONSECUTIVE_SITES = 4

res = 0
resj = []
tdiffj = []
i = 0
time_stamp = np.array(alice_df['timestamp'])
while i < len(time_stamp):
    j = 1
    while (i+j < len(time_stamp) and \
           (time_stamp[i+j]-time_stamp[i+j-1])/np.timedelta64(1, 's') <= TIME_DIFFERENCE):
        j += 1
    if j >= NUMBER_OF_CONSECUTIVE_SITES:
        resj.append(j)
        if i > 5:
            tdiffj.append(max([(time_stamp[i-t]-time_stamp[i-t-1])/np.timedelta64(1, 's') for t in range(5)]))
        else:
            tdiffj.append((time_stamp[i-t]-time_stamp[i-t-1])/np.timedelta64(1, 's'))
        res += 1
    i += j
print(np.mean(resj), res)
tdiffj

12.73370319001387 1442


[13.0,
 18.0,
 12.0,
 11.0,
 15.0,
 10.0,
 18.0,
 50.0,
 9.0,
 14.0,
 12.0,
 7.0,
 20.0,
 32.0,
 10.0,
 5.0,
 14.0,
 7.0,
 3.0,
 5.0,
 150.0,
 6.0,
 22.0,
 10.0,
 21.0,
 12.0,
 18.0,
 15.0,
 55.0,
 29.0,
 3.0,
 8.0,
 3.0,
 4.0,
 16.0,
 24.0,
 25.0,
 55.0,
 5.0,
 4.0,
 35.0,
 1565.0,
 3.0,
 19.0,
 5064229.0,
 28.0,
 4.0,
 11.0,
 10.0,
 5.0,
 10.0,
 14.0,
 6.0,
 6.0,
 28.0,
 22.0,
 7.0,
 8.0,
 70.0,
 4.0,
 20.0,
 49.0,
 42.0,
 42.0,
 13.0,
 19.0,
 4.0,
 10.0,
 3.0,
 13242574.0,
 166.0,
 10.0,
 20.0,
 8.0,
 8.0,
 3.0,
 3.0,
 9.0,
 6.0,
 6.0,
 19.0,
 10.0,
 6.0,
 10.0,
 5.0,
 7.0,
 9.0,
 3.0,
 3.0,
 7.0,
 7.0,
 15.0,
 4.0,
 3.0,
 5.0,
 6.0,
 6.0,
 6.0,
 3.0,
 5.0,
 4.0,
 5.0,
 8.0,
 5.0,
 3.0,
 6.0,
 6.0,
 7.0,
 10.0,
 14.0,
 6.0,
 5.0,
 4.0,
 4.0,
 8.0,
 3.0,
 5.0,
 70.0,
 7.0,
 7.0,
 4.0,
 4.0,
 6.0,
 3.0,
 3.0,
 5.0,
 3.0,
 10.0,
 4.0,
 8.0,
 3.0,
 3.0,
 3.0,
 8.0,
 4.0,
 5.0,
 4.0,
 6.0,
 7.0,
 8.0,
 3.0,
 8.0,
 7.0,
 7.0,
 3.0,
 3.0,
 5.0,
 12.0,
 9.0,
 3.0,
 13.0,
 6.0,
 14.0,
 15.0,

Гипотеза о том, что перед скоплениями сайтов обычно идут большие перерывы -- не сработала

In [223]:
resj

[3,
 4,
 3,
 8,
 12,
 14,
 8,
 3,
 4,
 9,
 3,
 10,
 11,
 7,
 10,
 5,
 5,
 14,
 7,
 7,
 5,
 6,
 9,
 7,
 8,
 8,
 8,
 5,
 5,
 3,
 8,
 3,
 5,
 8,
 5,
 15,
 11,
 6,
 7,
 3,
 3,
 11,
 15,
 5,
 5,
 3,
 5,
 3,
 5,
 11,
 11,
 4,
 6,
 5,
 11,
 9,
 6,
 9,
 4,
 5,
 3,
 3,
 11,
 10,
 4,
 9,
 3,
 8,
 4,
 5,
 3,
 5,
 3,
 4,
 7,
 3,
 13,
 11,
 11,
 4,
 6,
 4,
 4,
 3,
 5,
 5,
 7,
 5,
 3,
 4,
 6,
 3,
 3,
 8,
 4,
 3,
 3,
 13,
 4,
 6,
 9,
 5,
 7,
 5,
 4,
 4,
 6,
 5,
 9,
 5,
 3,
 7,
 9,
 3,
 3,
 4,
 4,
 4,
 5,
 4,
 6,
 3,
 5,
 3,
 3,
 3,
 7,
 8,
 9,
 4,
 4,
 3,
 3,
 4,
 5,
 5,
 7,
 9,
 4,
 5,
 3,
 3,
 3,
 4,
 5,
 3,
 7,
 6,
 4,
 4,
 4,
 5,
 4,
 4,
 3,
 4,
 8,
 6,
 5,
 3,
 5,
 5,
 6,
 7,
 8,
 4,
 4,
 4,
 7,
 8,
 8,
 5,
 6,
 6,
 7,
 4,
 6,
 10,
 3,
 6,
 3,
 5,
 6,
 7,
 7,
 3,
 4,
 3,
 3,
 6,
 6,
 4,
 3,
 3,
 3,
 5,
 4,
 5,
 5,
 3,
 4,
 3,
 8,
 4,
 5,
 3,
 5,
 3,
 5,
 5,
 4,
 4,
 3,
 4,
 5,
 7,
 4,
 5,
 4,
 5,
 7,
 5,
 4,
 3,
 4,
 7,
 4,
 4,
 3,
 4,
 3,
 3,
 6,
 3,
 5,
 7,
 3,
 5,
 4,
 5,
 5,
 7,
 3,
 6,
 3,


Был ли сайт открыт после определённого промежутка времени? Фича

In [11]:
test_df[times].count().sum()/(float)(test_df.shape[0]*10)

0.9451429399615928

Насколько плотно идут группы сайтов после долгого перерыва?

Мы должны создать сеть взаимоисключающих временных отрезков: когда есть пересечения интервалов времени внутри.

Возможен лидербоард пробинг, если интервалы не будут пересекаться вообще. Или как раз для непересекающихся интервалов.

Делаем sparse-matrix невозможных пересечений.

In [78]:
test_df = test_df.sort_values(by='time1')
t_start = np.array(test_df['time1'])
t_end = np.array(test_df['time10'])
for i, val in enumerate(t_end):
    if np.isnat(val):
        for col in times[-2::-1]:
            if not pd.isnull(test_df[col].iloc[i]):
                t_end[i] = test_df[col].iloc[i]
t_diff = (t_start[1:] - t_end[:-1])/np.timedelta64(1, 's')
len(t_diff[(0 <= t_diff) & (t_diff <= 1)])

20039

Похоже, 20к строк я смогу просто связать между собой без проблем. Короткие строки (мб меньше 5-7?) можно связывать с ближайшим вариантом, но в ограниченных пределах (меньше 15 секунд?).

In [130]:
different_users = (t_diff < 0)
len(different_users)

82796

Правило присоединения: время и количество пересечений сайтов!

Проанализируем, на сколько и какие строки мы даём шансы больше 0.5

In [167]:
answers = pd.read_csv('submission_alice_EyeShield77.csv', header=0, index_col='session_id')
answers.head()

,target
session_id,
1,0.000046
2,0.000005
3,0.000263
4,0.000004
5,0.000227


In [168]:
ch_pos = 0
ch_neg = 0
for i, yes in enumerate(res2):
    if yes:
        val1 = answers['target'].loc[indices[i+1]]
        val2 = answers['target'].loc[indices[i]]
        if val1 > 0.1 or val2 > 0.1:
            ch_pos += 1
            answers['target'].loc[indices[i+1]] = max(val1, val2)
            answers['target'].loc[indices[i]] = max(val1, val2)
        else:
            ch_neg += 1
            answers['target'].loc[indices[i+1]] = min(val1, val2)
            answers['target'].loc[indices[i]] = min(val1, val2)
print(ch_pos, ch_neg)

1274 14706


In [163]:
sum(yes)

TypeError: 'int' object is not iterable

In [160]:
np.sum(answers == 0)

target    104
dtype: int64

In [169]:
answers.to_csv(f'submission_alice_5.csv', index_label='session_id')

In [114]:
answers['target'][answers['target'] > 0.5]

session_id
84       0.505580
175      0.685069
333      0.691792
347      0.895003
426      0.836668
           ...   
82056    0.655291
82069    0.882539
82474    0.560028
82485    0.954359
82675    0.581654
Name: target, Length: 387, dtype: float64

In [145]:
answers.loc[64199]

target    0.000016
Name: 64199, dtype: float64

In [106]:
test_df.iloc[indices.index(1395)-15:indices.index(1395)+1]

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
32733,80,2014-06-01 11:05:16,80.0,2014-06-01 11:05:23,80.0,2014-06-01 11:05:26,80.0,2014-06-01 11:05:37,21.0,2014-06-01 11:06:02,21.0,2014-06-01 11:06:08,42411.0,2014-06-01 11:06:14,570.0,2014-06-01 11:06:14,42412.0,2014-06-01 11:06:14,42413.0,2014-06-01 11:06:14
10689,820,2014-06-01 11:05:19,820.0,2014-06-01 11:05:21,820.0,2014-06-01 11:05:22,820.0,2014-06-01 11:05:24,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
5775,21,2014-06-01 11:06:14,42412.0,2014-06-01 11:06:15,42411.0,2014-06-01 11:06:15,42414.0,2014-06-01 11:06:15,29.0,2014-06-01 11:06:15,42415.0,2014-06-01 11:06:15,42415.0,2014-06-01 11:06:16,42412.0,2014-06-01 11:06:33,29.0,2014-06-01 11:06:33,42416.0,2014-06-01 11:06:33
65694,42415,2014-06-01 11:06:33,42415.0,2014-06-01 11:06:34,42412.0,2014-06-01 11:06:34,42415.0,2014-06-01 11:06:36,29.0,2014-06-01 11:06:36,42412.0,2014-06-01 11:06:38,42415.0,2014-06-01 11:06:39,29.0,2014-06-01 11:06:39,42416.0,2014-06-01 11:06:39,42412.0,2014-06-01 11:06:39
80324,42412,2014-06-01 11:06:41,42411.0,2014-06-01 11:06:53,42411.0,2014-06-01 11:07:41,42411.0,2014-06-01 11:07:45,42412.0,2014-06-01 11:07:48,42412.0,2014-06-01 11:07:49,42415.0,2014-06-01 11:07:50,42412.0,2014-06-01 11:07:50,29.0,2014-06-01 11:07:50,76.0,2014-06-01 11:07:50
34610,21203,2014-06-01 11:07:14,21203.0,2014-06-01 11:07:17,21203.0,2014-06-01 11:07:18,21203.0,2014-06-01 11:07:21,21203.0,2014-06-01 11:07:53,21203.0,2014-06-01 11:08:30,21203.0,2014-06-01 11:08:32,21203.0,2014-06-01 11:08:33,21203.0,2014-06-01 11:08:48,21203.0,2014-06-01 11:08:50
24874,42416,2014-06-01 11:07:50,77.0,2014-06-01 11:07:51,42412.0,2014-06-01 11:07:53,42412.0,2014-06-01 11:07:54,29.0,2014-06-01 11:08:02,42416.0,2014-06-01 11:08:02,42412.0,2014-06-01 11:08:02,42415.0,2014-06-01 11:08:02,42415.0,2014-06-01 11:08:03,42412.0,2014-06-01 11:08:24
79875,42415,2014-06-01 11:08:24,29.0,2014-06-01 11:08:24,42416.0,2014-06-01 11:08:24,42415.0,2014-06-01 11:08:25,42415.0,2014-06-01 11:08:26,42412.0,2014-06-01 11:08:26,29.0,2014-06-01 11:08:26,42412.0,2014-06-01 11:08:27,42415.0,2014-06-01 11:08:27,42416.0,2014-06-01 11:08:27
20975,42417,2014-06-01 11:08:34,38.0,2014-06-01 11:08:34,19675.0,2014-06-01 11:08:34,23.0,2014-06-01 11:08:34,18082.0,2014-06-01 11:08:35,42417.0,2014-06-01 11:08:35,38.0,2014-06-01 11:08:35,19675.0,2014-06-01 11:08:35,23.0,2014-06-01 11:08:35,42414.0,2014-06-01 11:08:35


In [ ]:
def prepare_sparse_features(path_to_train, path_to_test, path_to_site_dict,
                           vectorizer_params):
    times = ['time%s' % i for i in range(1, 11)]
    train_df = pd.read_csv(path_to_train,
                       index_col='session_id', parse_dates=times)
    test_df = pd.read_csv(path_to_test,
                      index_col='session_id', parse_dates=times)

    # Sort the data by time
    train_df = train_df.sort_values(by='time1')
    
    # read site -> id mapping provided by competition organizers 
    with open(path_to_site_dict, 'rb') as f:
        site2id = pickle.load(f)
    # create an inverse id _> site mapping
    id2site = {v:k for (k, v) in site2id.items()}
    # we treat site with id 0 as "unknown"
    id2site[0] = 'unknown'
    
    # Transform data into format which can be fed into TfidfVectorizer
    # This time we prefer to represent sessions with site names, not site ids. 
    # It's less efficient but thus it'll be more convenient to interpret model weights.
    sites = ['site%s' % i for i in range(1, 11)]
    train_sessions = train_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    test_sessions = test_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    # we'll tell TfidfVectorizer that we'd like to split data by whitespaces only 
    # so that it doesn't split by dots (we wouldn't like to have 'mail.google.com' 
    # to be split into 'mail', 'google' and 'com')
    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_sessions)
    X_test = vectorizer.transform(test_sessions)
    y_train = train_df['target'].astype('int').values
    
    # we'll need site visit times for further feature engineering
    train_times, test_times = train_df[times], test_df[times]
    
    return X_train, X_test, y_train, vectorizer, train_times, test_times

In [58]:
# Sort the data by time
train_df = train_df.sort_values(by='time1')
test_df = test_df.sort_values(by='time1')

In [185]:
test_df = test_df.sort_values(by='time1')
t_start = np.array(test_df['time1'])
t_end = np.array(test_df['time10'])
for i, val in enumerate(t_end):
    if np.isnat(val):
        for col in times[-2::-1]:
            if not pd.isnull(test_df[col].iloc[i]):
                t_end[i] = test_df[col].iloc[i]
t_diff = (t_start[1:] - t_end[:-1])/np.timedelta64(1, 's')
len(t_diff[(0 <= t_diff) & (t_diff <= 1)])

20011

In [187]:
#different t_diff!!!
t_diff = (t_start[1:] - np.array(test_df['time10'])[:-1])/np.timedelta64(1, 's')
len(t_diff[(0 <= t_diff) & (t_diff <= 1)])

18967

In [188]:
sites = ['site%d' % i for i in range(1, 11)]
s = np.array(test_df[sites])
common = []
for i in range(1, len(s)):
    common.append(set(s[i-1]).intersection(set(s[i])))
concat = [1 if len(x) >= 2 and (0 <= y <= 3) else 0 for x, y in zip(common, t_diff)]
sum(concat)

15771

In [68]:
indices = test_df.index.tolist()

In [70]:
short_ones = test_df[test_df['time7'].isnull()].index.tolist()
len(short_ones)

5957

In [79]:
for id in short_ones[:5]:
    i = indices.index(id)
    while i > 0:
        
    print(t_start[indices.index(id)])
    

2014-05-01T17:14:03.000000000
2014-05-02T07:52:08.000000000
2014-05-02T07:57:51.000000000
2014-05-02T08:07:52.000000000
2014-05-02T08:08:00.000000000


In [117]:
test_df[:30]

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
65540,21,2014-05-01 17:14:03,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
64199,23,2014-05-02 07:52:08,66.0,2014-05-02 07:54:08,63.0,2014-05-02 07:54:08,2626.0,2014-05-02 07:55:09,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2268,979,2014-05-02 07:57:51,73.0,2014-05-02 07:59:34,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
29734,66,2014-05-02 08:05:16,69.0,2014-05-02 08:05:17,67.0,2014-05-02 08:05:17,70.0,2014-05-02 08:05:17,71.0,2014-05-02 08:05:17,68.0,2014-05-02 08:05:17,71.0,2014-05-02 08:05:18,70.0,2014-05-02 08:05:18,69.0,2014-05-02 08:05:18,67.0,2014-05-02 08:05:18
77048,167,2014-05-02 08:05:32,167.0,2014-05-02 08:05:33,359.0,2014-05-02 08:05:34,167.0,2014-05-02 08:05:34,167.0,2014-05-02 08:05:35,305.0,2014-05-02 08:09:19,306.0,2014-05-02 08:09:20,306.0,2014-05-02 08:09:22,979.0,2014-05-02 08:09:54,68.0,2014-05-02 08:12:46
33526,67,2014-05-02 08:06:03,70.0,2014-05-02 08:06:03,69.0,2014-05-02 08:06:03,68.0,2014-05-02 08:06:03,71.0,2014-05-02 08:06:03,66.0,2014-05-02 08:06:03,67.0,2014-05-02 08:06:04,69.0,2014-05-02 08:06:04,69.0,2014-05-02 08:06:06,70.0,2014-05-02 08:06:07
61453,69,2014-05-02 08:06:07,167.0,2014-05-02 08:06:08,67.0,2014-05-02 08:06:09,71.0,2014-05-02 08:06:09,70.0,2014-05-02 08:06:09,23.0,2014-05-02 08:06:25,21.0,2014-05-02 08:06:25,21.0,2014-05-02 08:06:40,22.0,2014-05-02 08:06:42,23.0,2014-05-02 08:06:43
17228,23,2014-05-02 08:06:40,66.0,2014-05-02 08:08:40,63.0,2014-05-02 08:08:40,21.0,2014-05-02 08:09:30,21.0,2014-05-02 08:09:31,22.0,2014-05-02 08:09:32,23.0,2014-05-02 08:09:32,797.0,2014-05-02 08:09:37,22.0,2014-05-02 08:09:41,23.0,2014-05-02 08:09:41
44868,69,2014-05-02 08:06:46,71.0,2014-05-02 08:06:46,68.0,2014-05-02 08:06:46,66.0,2014-05-02 08:06:46,70.0,2014-05-02 08:06:46,67.0,2014-05-02 08:06:46,67.0,2014-05-02 08:06:47,69.0,2014-05-02 08:06:47,70.0,2014-05-02 08:06:47,71.0,2014-05-02 08:06:48


In [1]:
word = 'kdjfj'
type(word)

str